In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-w6p2/X_test.csv
/kaggle/input/2022-ai-w6p2/train.csv
/kaggle/input/2022-ai-w6p2/sample_submit.csv


In [2]:
import random
import torch

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
train = pd.read_csv('/kaggle/input/2022-ai-w6p2/train.csv')
x_test = pd.read_csv('/kaggle/input/2022-ai-w6p2/X_test.csv')
submit = pd.read_csv('/kaggle/input/2022-ai-w6p2/sample_submit.csv')

In [4]:
train.columns

Index(['Day', 'Hour', 'Minute', 'DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET'], dtype='object')

In [5]:
x_train = train[['Day', 'Hour', 'Minute', 'DHI', 'DNI', 'WS', 'RH', 'T']]
y_train = train['TARGET']

In [6]:
x_test = x_test.drop(['ID'], axis=1)

In [7]:
x_test

,Day,Hour,Minute,DHI,DNI,WS,RH,T
0,1043,7,30,34,584,2.0,63.02,3
1,1043,8,0,45,730,2.3,61.54,5
2,1043,8,30,52,817,2.4,57.34,6
3,1043,9,0,57,878,2.6,52.46,8
4,1043,9,30,61,917,2.2,45.74,10
...,...,...,...,...,...,...,...,...
2476,1094,21,30,0,0,2.4,70.70,-4
2477,1094,22,0,0,0,2.4,66.79,-4
2478,1094,22,30,0,0,2.2,66.78,-4
2479,1094,23,0,0,0,2.1,67.72,-4


In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [9]:
x_train = torch.FloatTensor(x_train).cuda()
y_train = torch.FloatTensor(y_train).cuda()
x_test = torch.FloatTensor(x_test).cuda()

In [10]:
layer1 = torch.nn.Linear(8, 64, bias = True)
layer2 = torch.nn.Linear(64, 64, bias = True)
layer3 = torch.nn.Linear(64, 64, bias = True)
layer4 = torch.nn.Linear(64, 1, bias = True)
sigmoid = torch.nn.Sigmoid()
model = torch.nn.Sequential(layer1, sigmoid, layer2, sigmoid, layer3, sigmoid, layer4).cuda()

In [11]:
loss = torch.nn.MSELoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [12]:
y_train = y_train.reshape([9999,1])

In [13]:
for stop in range(1001):
    optimizer.zero_grad()
    hypothesis = model(x_train)
    cost = loss(hypothesis, y_train)
    cost.backward()
    optimizer.step()
    if stop % 100 == 0:
        print(stop, cost.item())

0 1032.979248046875
100 844.031494140625
200 754.2284545898438
300 710.4535522460938
400 689.4500732421875
500 542.1607055664062
600 408.0322570800781
700 318.5309143066406
800 255.77232360839844
900 209.43031311035156
1000 172.200439453125


In [14]:
predict = model(x_test)
predict[1000:1020]

tensor([[ 4.1024],
        [ 4.0539],
        [ 4.1221],
        [ 4.0557],
        [ 4.1289],
        [ 4.0610],
        [ 4.0949],
        [ 4.0899],
        [11.5988],
        [17.2653],
        [21.5407],
        [24.8398],
        [27.2742],
        [29.4299],
        [30.5241],
        [29.8702],
        [28.0523],
        [14.1280],
        [22.6239],
        [28.3245]], device='cuda:0', grad_fn=<SliceBackward0>)

In [15]:
submit['TARGET'] = predict.cpu().detach().numpy()
submission=submit.astype(np.str_)
submission.to_csv('submission.csv',mode='w',header=True,index=False)